# **Introdução**


# **Análise Exploratória dos Dados**

##### Importação das bibliotecas


In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import warnings

##### Carregamento das bases de treino e teste

In [ ]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv') 

#### Estrutura do Dataset

**Análise Descritiva**


In [ ]:
train_data.describe()

A tabela acima mostra as estatísticas descritivas do conjunto de dados. Algumas informações importantes são: 

- **Idade e marcos:** as colunas de idade em relação ao primeiro/último financiamento e ao primeiro/último marco apresentam valores médios entre 2 e 5 anos, mas com grande variação (máximo chegando a mais de 20 anos). Isso indica que existem tanto startups muito jovens quanto outras mais consolidadas.
- **Relacionamentos e rodadas de investimento:** em média, cada empresa tem cerca de 8 relacionamentos e 2,3 rodadas de financiamento. O máximo observado foi de 63 relacionamentos e 8 rodadas, mostrando casos de alto networking e captação recorrente.
- **Financiamento total:** há uma grande dispersão em funding_total_usd. O valor médio é em torno de 29 milhões de dólares, mas o desvio padrão é muito alto, chegando a um máximo de 5,7 bilhões. Isso sugere forte assimetria e presença de outliers.
- **Milestones:** a média de marcos atingidos é próxima de 2, com máximo de 6.
- **Variáveis binárias:** indicam se a empresa pertence a determinada categoria ou atingiu uma rodada específica. A média desses campos mostra a proporção de empresas que se enquadram em cada caso. Por exemplo, cerca de 55% são da Califórnia (is_CA), 33% receberam investimento de venture capital (has_VC) e 51% chegaram à rodada A (has_roundA).
- **Participantes médios:** a média é de 2,8 investidores por rodada, com máximo de 16.
- **Labels:** aproximadamente 65% das observações têm rótulo positivo (1).

Em resumo, os dados revelam grande heterogeneidade entre as empresas, especialmente no volume de financiamento. Também fica claro que uma parcela significativa das startups passou por rodadas iniciais de captação (seed e A), mas poucas alcançaram rodadas mais avançadas (C ou D).

**Informações do Dataset**

In [ ]:
train_data.info()

AO dataset possui 646 entradas e 33 colunas. A maior parte das variáveis é numérica (`int64` ou `float64`), representando quantidades (ex.: número de rodadas de investimento, relacionamentos, marcos) ou indicadores binários (ex.: "is_CA", "has_VC"). Apenas a coluna "category_code" está no formato `object`, pois representa categorias de empresas em texto. Também é possível notar que algumas colunas do tipo `float64` possuem valores nulos, como "age_first_funding_year", "age_last_funding_year" e "age_first_milestone_year", o que pode exigir tratamento posterior.

**Registros de Amostra**

In [ ]:
train_data.head()

**Dimensões e Estrutura do Dataset**

In [ ]:
train_data.shape

In [ ]:
train_data.columns

- **id** (`int64`) → Identificador único da startup no dataset.  

**Variável alvo**
- **labels** (`int64`) → Indicador de sucesso:  
  - **1** = startup bem-sucedida (ativa/adquirida).  
  - **0** = startup fechada/insucesso.  
  - **Distribuição**: ~65% sucesso, ~35% insucesso.  

**Idades relativas (anos desde a fundação até o evento)** 
- **age_first_funding_year** (`float64`) → Anos até o primeiro funding (≥ 0 ou NaN).  
- **age_last_funding_year** (`float64`) → Anos até o último funding (≥ 0 ou NaN).  
- **age_first_milestone_year** (`float64`) → Anos até o primeiro marco relevante (muitos NaN).  
- **age_last_milestone_year** (`float64`) → Anos até o último marco relevante (muitos NaN).  

**Estrutura, histórico e escala de captação**
- **relationships** (`int64`) → Número de relacionamentos (fundadores, executivos, investidores).  
- **funding_rounds** (`int64`) → Número de rodadas de captação.  
- **funding_total_usd** (`float64`) → Total captado em dólares (valores extremos tratados).  
- **milestones** (`int64`) → Quantidade de marcos relevantes registrados.  
- **avg_participants** (`float64`) → Média de investidores por rodada de funding.  

**Localização (variáveis binárias)** 
- **is_CA**, **is_NY**, **is_MA**, **is_TX**, **is_otherstate** (`int64`) → Indicam o estado-sede da startup (Califórnia, Nova Iorque, Massachusetts, Texas ou Outros).  

**Setor/mercado**
- **category_code** (`object`) → Setor principal declarado da startup (string).  
- **is_software**, **is_web**, **is_mobile**, **is_enterprise**, **is_advertising**, **is_gamesvideo**, **is_ecommerce**, **is_biotech**, **is_consulting**, **is_othercategory** (`int64`) → Indicadores de setor em formato binário (0/1).  

**Sinalizadores de financiamento**
- **has_VC** (`int64`) → Recebeu **venture capital**? (0/1).  
- **has_angel** (`int64`) → Recebeu **investimento anjo**? (0/1).  
- **has_roundA**, **has_roundB**, **has_roundC**, **has_roundD** (`int64`) → Indicam se a startup chegou até cada rodada de funding específica.  

In [ ]:
train_data['labels'].value_counts()

In [ ]:
print(f"Taxa de sucesso: {train_data['labels'].mean():.2%}")